build_dataset 해당

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData

macro_agent = MacroAData()
macro_agent.fetch_data()
macro_agent.add_features()
macro_agent.save_csv()
macro_agent.make_close_price()

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData
# macro_agent = MacroAData()
macro_agent.model_maker()

$$$$   X = macro_sercher(agent, ticker) 역할

In [1]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료


In [2]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed

[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)
[OK] 매크로 데이터 수집 완료: (42, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료


In [3]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

X_seq: [[[-0.39995147 -0.40595861  1.09840824 ... -0.10168135  3.62480943
    3.69374719]
  [-0.38305979 -0.45630697  1.16231301 ... -0.56604179  3.62480943
    3.69374719]
  [-0.31946784 -0.58130828  1.34705066 ... -0.86378545  3.62480943
    3.69374719]
  ...
  [-0.52614229  0.03675882  0.68499278 ... -0.60638168  4.30134133
    4.23632084]
  [-0.56340335  0.02460586  0.70425175 ... -1.16676943  4.22321708
    4.25204222]
  [-0.56489373 -0.03615894  0.79423877 ... -0.07910566  4.14757206
    4.25706138]]]
X_scaled:     Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X  Open_GC=F  Open_HG=F  Open_QQQ  \
0   -0.358716      -0.378180       1.058849   6.373568   1.151408  3.065970   
1   -0.434728      -0.425056       1.112574   6.458651   1.213238  3.089109   
2   -0.399951      -0.405959       1.098408   6.400298   1.230676  3.132817   
3   -0.383060      -0.456307       1.162313   6.556156   1.286164  3.186422   
4   -0.319468      -0.581308       1.347051   6.509474   1.183115  3.167011   
5  

$$$$   macro_predictor(X) 역할

In [4]:
pred_prices, target = macro_sub.m_predictor(X_seq)

[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
NVDA: 마지막 종가=198.20 → 예측 종가=198.50 (예상 수익률 0.15%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.195001       198.496743          0.001522     0.152245   

   uncertainty  confidence  
0     0.050787   19.690245  

================= 예측 결과 (값) =================
{'NVDA': 198.49674292529164}


In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [5]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)

3️⃣ Running GradientAnalyzer for interpretability...
[INFO] Running Gradient × Input + Integrated Gradients analysis...
[DEBUG] Gradient × Input output shape: (1, 42, 169)
[DEBUG] x_input shape before IG: (1, 42, 169)
[DEBUG] interpolated shape: (51, 42, 169)
[INFO] IG–G×I correlation (agreement_ratio): 0.8002
[INFO] Gradient analysis completed successfully.

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 198.49674292529164} =================

================= LLM Explanation =================
(1) Feature Trend 분석: 최근 시점에서 'Close_GC=F'와 'Open_GC=F'의 영향력이 증가하였으며, 이는 금 선물 가격이 반도체 산업과 관련된 원자재 가격에 미치는 영향이 커지고 있음을 시사합니다. 반면 일부 변동성이 낮은 변수들은 영향력이 상대적으로 약화되었습니다. 이러한 변화는 글로벌 경기 불확실성과 원자재 시장 변동성 확대라는 거시적 요인과 연관됩니다. 시간에 따른 변수 영향력 증가는 모델이 금 가격 변동을 통해 NVDA 주가를 예측하는 데 있어 원자재 가격의 중요성이 커졌음을 의미합니다.

(2) Model C

In [ ]:
import numpy as np

# print("원본 X 평균:", np.mean(X, axis=0)[:10])
print("스케일링 후 X 평균:", np.mean(X_scaled, axis=0)[:10])
print("스케일링 후 X 표준편차:", np.std(X_scaled, axis=0)[:10])
print("pred_prices:", pred_prices)


In [6]:
print(macro_sub.stockdata.MacroSentiAgent["feature_importance"])


{'feature_summary': {'agreement_ratio': 0.800167520800897, 'gx_importance_top': ['Close_GC=F', 'Low_GC=F', 'High_^GSPC'], 'ig_importance_top': ['Open_GC=F', 'Close_GC=F', 'High_GC=F']}, 'importance_dict': {np.str_('Open_CL=F'): 9.606967068975791e-05, np.str_('Open_DX-Y.NYB'): 5.108551340526901e-05, np.str_('Open_EURUSD=X'): 0.00026529404567554593, np.str_('Open_GC=F'): 0.0017100374680012465, np.str_('Open_HG=F'): 0.0004287340852897614, np.str_('Open_QQQ'): 0.00045406303252093494, np.str_('Open_SPY'): 0.000425292004365474, np.str_('Open_USDJPY=X'): 0.0002589369541965425, np.str_('Open_^DJI'): 0.0008704881183803082, np.str_('Open_^FVX'): 0.000238278487813659, np.str_('Open_^GSPC'): 0.00045495101949200034, np.str_('Open_^IRX'): 7.796093996148556e-05, np.str_('Open_^IXIC'): 0.0003789400216192007, np.str_('Open_^TNX'): 0.00016797136049717665, np.str_('Open_^VIX'): 0.0001332883257418871, np.str_('High_CL=F'): 0.00015933216491248459, np.str_('High_DX-Y.NYB'): 3.6333585740067065e-05, np.str_('

In [8]:
print(f"total_json: {total_json}")

total_json: {'agent_id': 'MacroSentiAgent', 'target': Target(next_close=198.49674292529164, uncertainty=0.05078655853867531, confidence=19.690244674682617), 'reason': "(1) Feature Trend 분석: 최근 시점에서 'Close_GC=F'와 'Open_GC=F'의 영향력이 증가하였으며, 이는 금 선물 가격이 반도체 산업과 관련된 원자재 가격에 미치는 영향이 커지고 있음을 시사합니다. 반면 일부 변동성이 낮은 변수들은 영향력이 상대적으로 약화되었습니다. 이러한 변화는 글로벌 경기 불확실성과 원자재 시장 변동성 확대라는 거시적 요인과 연관됩니다. 시간에 따른 변수 영향력 증가는 모델이 금 가격 변동을 통해 NVDA 주가를 예측하는 데 있어 원자재 가격의 중요성이 커졌음을 의미합니다.\n\n(2) Model Consistency 분석: Integrated Gradients와 Gradient×Input 결과가 일치하는 주요 변수는 'Close_GC=F', 'Open_GC=F', 'High_SPY' 등이며, 이는 모델이 신뢰할 만한 구조적 요인을 반영하고 있음을 보여줍니다. 반면, 'Open_DX-Y.NYB', 'Open_EURUSD=X' 등은 일관성 차이가 커서 시장 불확실성, 데이터 잡음, 또는 비선형적 상호작용 가능성을 내포합니다.\n\n(3) Sensitivity 분석: 'Close_GC=F'와 'Low_GC=F'는 입력 변화에 민감도가 높아 단기 시장 변동성에 크게 반응합니다. 이는 단기 트레이딩 전략에서 리스크 관리가 필요함을 시사하며, 포트폴리오 내 변동성 관리에 중요한 변수입니다. 반면 민감도가 낮은 변수들은 상대적으로 안정적인 장기적 요인을 반영합니다.\n\n(4) Stability 분석: 'Open_GC=F'와 'High_GC=F'는 중요도 변동성이 낮아 구조적이고 장기적인 트렌드를 반영하는 반면, 변동성이 높은 변

In [7]:
print(f"opinion: {opinion}")

opinion: Opinion(agent_id='MacroSentiAgent', target=Target(next_close=198.49674292529164, uncertainty=0.05078655853867531, confidence=19.690244674682617), reason='기술적 분석 측면에서, 금 선물(Close_GC=F)의 종가와 저점(Low_GC=F)이 높은 중요도를 보이며, 이는 최근 이동평균선 상향과 거래량 증가, 변동성 완화, RSI 지표의 중립 이상 신호와 맞물려 긍정적 흐름을 시사한다. 또한, S&P 500 지수(High_SPY)의 고점 상승도 기술적 강세를 뒷받침한다. 거시경제적으로는 금리 상승 압력이 완화되고, 인플레이션 안정화 조짐과 함께 환율 변동성도 제한적이며, 유동성 환경은 완화되는 정책 방향과 일치하여 위험자산 선호 심리를 강화한다. Gradient 기반 해석에서는 Integrated Gradients가 Open_GC=F, Close_GC=F, High_GC=F를 주요 긍정 변수로 지목하며, Gradient×Input도 Close_GC=F, Low_GC=F, High_^GSPC를 중요 변수로 꼽아 두 기법 간 일관성이 높다. 다만 Open_GC=F의 순위 변동성(rank_gap)이 다소 존재해 단기 변동성 요인으로 작용할 수 있다. 민감도 분석 결과 Close_GC=F와 Low_GC=F가 높은 민감도를 보여 단기 리스크 요인으로 작용하는 반면, Open_GC=F와 High_SPY는 안정적인 변수로 구조적 상승 추세를 지지한다. 종합하면, 기술적 지표와 거시경제 환경, Gradient 해석 결과가 모두 긍정적 신호를 내포하고 있어 다음 거래일 종가는 상승할 가능성이 높다. 특히 금 선물과 주요 지수의 강세가 투자 심리를 견인하며, 단기 변동성은 있으나 전반적 상승 모멘텀을 유지할 것으로 판단된다. 따라서 예측 종가는 198.497로 제시하며, 이는 데이터 기반의 다각적 분석에 근거한 합리적 전망이다. \xa0이러한 분

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [9]:
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [10]:
debate_agent.run_dataset()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
✅ NVDA TechnicalAgent dataset saved via technical_data_set
[TRACE B] macro_dataset() start for NVDA


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Data shape: (1304, 90), Columns: 90
[TRACE A] add_features() for self.data:            Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X    Open_GC=F  Open_HG=F  \
Date                                                                          
2020-01-01        NaN            NaN       1.122083          NaN        NaN   
2020-01-02  61.599998      96.480003       1.121894  1518.099976     2.8165   
2020-01-03  61.180000      96.790001       1.117081  1530.099976     2.7935   
2020-01-06  63.709999      96.900002       1.116246  1580.000000     2.7780   
2020-01-07  62.910000      96.650002       1.119583  1558.300049     2.8010   
...               ...            ...            ...          ...        ...   
2024-12-24  69.559998     108.160004       1.040583  2613.000000     4.0525   
2024-12-25        NaN            NaN       1.040258          NaN        NaN   
2024-12-26  70.199997     108.169998       1.039955  2628.500000     4.0730   
2024-12-27  69.680000     108.0800

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
[INFO] Removed all constant Volume columns matching patterns: ['Volume_^FVX', 'Volume_^IRX', 'Volume_^TNX', 'Volume_^VIX', 'Volume_DX-Y.NYB', 'Volume_EURUSD=X', 'Volume_USDJPY=X']
[INFO] Remaining Volume columns: ['Volume_CL=F', 'Volume_GC=F', 'Volume_HG=F', 'Volume_QQQ', 'Volume_SPY', 'Volume_^DJI', 'Volume_^GSPC', 'Volume_^IXIC', 'Volume_CL=F_ret', 'Volume_GC=F_ret', 'Volume_HG=F_ret', 'Volume_QQQ_ret', 'Volume_SPY_ret', 'Volume_^DJI_ret', 'Volume_^GSPC_ret', 'Volume_^IXIC_ret']
[INFO] 병합 후 데이터 shape: (1257, 271)
Epoch 1/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - loss: 0.1384 - val_loss: 0.0908
Epoch 2/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.1301 - val_loss: 0.0991
Epoch 3/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 0.1273 - val_loss: 0.0822
Epoch 4/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.1288 - val_loss: 0.0900
Epoch 5/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.1252 - val_loss: 0.0849
Epoch 6/6

In [11]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


TechnicalAgent: <class 'str'> StandardScaler
NVDA
MacroSentiAgent: <class 'str'> StandardScaler
NVDA
SentimentalAgent: <class 'str'> StandardScaler
NVDA


In [12]:
opinions = debate_agent.get_opinion(0, "NVDA")

[TechnicalAgent] searcher 실행
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] pretrain 실행
[16:17:07] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.386084
  Epoch 010 | Loss: 0.392831
  Epoch 015 | Loss: 0.391654
  Epoch 020 | Loss: 0.380180
  Epoch 025 | Loss: 0.384888
  Epoch 030 | Loss: 0.383998
  Epoch 035 | Loss: 0.383461
  Epoch 040 | Loss: 0.379059
  Epoch 045 | Loss: 0.385918
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt
[TechnicalAgent] predict 실행
[TechnicalAgent] reviewer_draft 실행
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
  - TechnicalAgent: next_close=188.7723
MacroSentiAgent의 데이터 로드.. macro_s

[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


...

[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)
[OK] 매크로 데이터 수집 완료: (42, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
■ macro_sercher StockData 생성 완료 (NVDA)
MacroSentiAgent의 예측
[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
NVDA: 마지막 종가=198.20 → 예측 종가=201.62 (예상 수익률 1.73%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.195001       201.622755          0.017295     1.729486   

   uncertainty  confidence  
0     0.048332   20.690252  

================= 예측 결과 (값) =================
{'NVDA': 201.62275518478705}
[MacroSentiAgent] LLM (macro_reviewer_draft)
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)

3️⃣ Running GradientAnalyzer for interpretability...
[INFO] Running Gradient × Input + Integrated Gradients analysis...
[DEBUG] Gradient × Input output shape: (1, 42, 169)
[DEBUG] x_input shape before IG: (1, 42, 169)
[DEBUG] interpolated shape: (51, 42, 169)
[INFO] IG–G×I correlation (agreement_ratio): 0.8402
[INFO] Gradient analysis completed successfully.

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 201.62275518478705} =================

================= LLM Explanation =================
(1) Feature Trend 분석: 'Open_GC=F'와 'High_GC=F'의 영향력이 최근 시점으로 갈수록 커졌으며, 이는 금 선물 가격의 상승세가 반영된 것으로 보입니다. 반면 일부 기술주 지수 관련 변수는 상대적으로 영향력이 약화되었습니다. 이는 금 가격 상승과 기술주 변동성 간의 상반된 시장 흐름과 연관됩니다. 이러한 변화는 금 가격 상승이 NVDA 주가에 긍정적 영향을 미칠 수 있음을 시사합니다.

(2) Model Consistency 분석: 'Open_GC=F'와 'High_GC=F'는 Integrated Gradients

In [13]:
print(f"opinions:{opinions}")

opinions:{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=188.7723, uncertainty=0.0052, confidence=0.9948, idea={'per_time': [{'date': "['2025-06-03', '2025-06-04', '2025-06-05', '2025-06-06', '2025-06-09', '2025-06-10', '2025-06-11', '2025-06-12', '2025-06-13', '2025-06-16', '2025-06-17', '2025-06-18', '2025-06-20', '2025-06-23', '2025-06-24', '2025-06-25', '2025-06-26', '2025-06-27', '2025-06-30', '2025-07-01', '2025-07-02', '2025-07-03', '2025-07-07', '2025-07-08', '2025-07-09', '2025-07-10', '2025-07-11', '2025-07-14', '2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-21', '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25', '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31', '2025-08-01', '2025-08-04', '2025-08-05', '2025-08-06', '2025-08-07', '2025-08-08', '2025-08-11', '2025-08-12', '2025-08-13', '2025-08-14', '2025-08-15', '2025-08-18', '2025-08-19', '2025-08-20']", 'sum_abs': [0.018785262618735706, 0.018117282821603898, 0.017

In [14]:
print(f"opinions:{opinions.get('MacroSentiAgent')}")

opinions:Opinion(agent_id='MacroSentiAgent', target=Target(next_close=201.62275518478705, uncertainty=0.04833192750811577, confidence=20.69025230407715), reason='1. 기술적 분석 측면에서 NVDA 주가는 최근 이동평균선 위에 위치하며, 거래량도 안정적으로 유지되고 있어 긍정적인 모멘텀을 시사한다. 변동성은 중간 수준으로, RSI 지표는 과매수 구간에 진입하지 않아 추가 상승 여력이 존재한다. 2. 거시경제적으로는 금리 상승 압력이 완화되고, 인플레이션 둔화 조짐과 함께 달러 환율도 안정세를 보이고 있다. 이는 유동성 환경 개선과 정책 방향의 완화 기대감을 반영하여 주식시장에 긍정적 영향을 미친다. 3. Gradient 기반 해석에서는 Open_GC=F(금 선물 시가)와 High_GC=F(금 선물 고가), 그리고 Adj Close_^IXIC(나스닥 지수 조정 종가)가 주요 변수로 나타났다. Integrated Gradients와 Gradient×Input 모두에서 Open_GC=F가 강한 긍정적 영향력을 보이며 일관성이 높다. 다만 일부 변수는 순위 차이가 존재해 세부 영향력에는 변동성이 있다. 4. 민감도 분석 결과 Open_GC=F와 Volume_^IXIC(나스닥 거래량), Low_^IXIC(나스닥 저가) 등이 높은 민감도를 보여 단기 리스크 요인으로 작용할 수 있으나, Adj Close_^IXIC와 Open_GC=F는 안정적인 변수로 구조적 상승 트렌드를 지지한다. 5. 종합하면, 기술적 지표의 긍정적 신호와 거시경제 환경의 완화, 그리고 주요 변수들의 일관된 긍정적 영향력에 근거하여 다음 거래일 NVDA 종가는 상승할 가능성이 높다. 단기 변동성은 존재하나 전반적인 금융 및 시장 심리적 근거가 상승 전망을 뒷받침한다.')


리뷰탈 해보기..

In [15]:
debate_agent.get_rebuttal(1)

ValueError: TechnicalAgent 데이터 구조 오류: dict형 컬럼 데이터가 필요함

get_revise 해보기

In [ ]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()